In [1]:
# required libraries
import openai
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [23]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
loader = PyPDFLoader("data/Zivilgesetzbuch.pdf")
chunks = loader.load_and_split(text_splitter)

In [24]:
print(chunks[0].page_content)
print("The chunk contains " + str(len(chunks[0].page_content)) + " characters")

1 / 384 Schweizerisches Zivilgesetzbuch   
vom 10. Dezember 1907  (Stand am 1. Januar 2023)  
 
Die Bundesversammlung  der Schweizerischen Eidgenossenschaft,  
gestützt auf Artikel 64 der Bundesverfassung1,2  
nach Einsicht in eine Botschaft des Bundesrates vom 28. Mai 19043, 
beschliesst:  
Einleitung  
Art. 1  
1 Das Gesetz findet auf alle Rechtsfragen Anw endung, für die es nach 
Wortlaut oder Auslegung eine Bestimmung enthält.  
2 Kann dem Gesetz keine Vorschrift entnommen werden, so soll das 
Gericht4 nach Gewohnheitsrecht und, wo auch ein solches fehlt, nach 
der Regel entscheiden, die es als Gesetzgeber aufs tellen würde.  
3 Es folgt dabei bewährter Lehre und Überlieferung.  
Art. 2  
1 Jedermann hat in der Ausübung seiner Rechte und in der Erfüllung 
seiner Pflich ten nach Treu und Glauben zu handel n. 
2 Der offenbare Missbrauch eines Rechtes findet keinen Rechts schutz.    
Art. 3  
1 Wo das Gesetz eine Rechtswirkung an den guten Glauben einer Per -
The chunk contains 974 ch

In [25]:
# Load environment variables from .env file
load_dotenv()

# Access the API key using the variable name defined in the .env file
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI chat model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.8)

# initialize the OpenAI embeddings model
embeddings = OpenAIEmbeddings()

In [27]:
if os.path.exists("zivillaw_db"):
    print("Loading zivillaw_db from disk...")
    Chroma(persist_directory="zivillaw_db", embedding_function=embeddings)
else:
    chroma_db = Chroma.from_documents(documents=chunks,
                                    embedding=embeddings,
                                    persist_directory="zivillaw_db",
                                    collection_name="lc_chroma_demo")

In [28]:
query = "What is this document about?"

In [29]:
result = chroma_db.similarity_search(query)
print(result)

[Document(page_content='Papier -Schuldbrief zudem aus dem Titel ergeben.  III. Stellung des \nEigentümers  \nIV. Veräusse-\nrung. Teilung  \nV. Schuldbrief-\nforderung und \nNebenverein -\nbarungen  \n1. Im Allgemei-\nnen \n2. Kündigung  \nVI. Schutz des \nguten Glaubens  \nVII. Einreden \ndes Schuldners', metadata={'page': 256, 'source': 'data/Zivilgesetzbuch.pdf'}), Document(page_content='Schweizerisches Zivilgesetzbuch  \n270 / 384 210 \nArt. 925  \n1 Werden für Waren, die einem Frachtführer oder einem Lagerhaus \nübergeben sind, Wertpapiere ausgestellt, die sie vertreten, so gilt die \nÜbertragung einer solchen Urkunde als Übertragung der Ware selbst.  \n2 Steht jedoch dem gutgläubigen Empfänger des Warenpapiers ein gut -\ngläubiger Empfänger der Ware gegenüber, so geht dieser jenem vor.  \nArt. 926  \n1 Jeder Besitzer darf sich verbotener Eig enmacht mit Gewalt erweh ren. \n2 Er darf sich, wenn ihm die Sache durch Gewalt oder heimlich ent zo-\ngen wird, sofort des Grundstückes dur

In [30]:
result_with_scores = chroma_db.similarity_search_with_score(query)
print(result_with_scores)

[(Document(page_content='Papier -Schuldbrief zudem aus dem Titel ergeben.  III. Stellung des \nEigentümers  \nIV. Veräusse-\nrung. Teilung  \nV. Schuldbrief-\nforderung und \nNebenverein -\nbarungen  \n1. Im Allgemei-\nnen \n2. Kündigung  \nVI. Schutz des \nguten Glaubens  \nVII. Einreden \ndes Schuldners', metadata={'page': 256, 'source': 'data/Zivilgesetzbuch.pdf'}), 0.4186582565307617), (Document(page_content='Schweizerisches Zivilgesetzbuch  \n270 / 384 210 \nArt. 925  \n1 Werden für Waren, die einem Frachtführer oder einem Lagerhaus \nübergeben sind, Wertpapiere ausgestellt, die sie vertreten, so gilt die \nÜbertragung einer solchen Urkunde als Übertragung der Ware selbst.  \n2 Steht jedoch dem gutgläubigen Empfänger des Warenpapiers ein gut -\ngläubiger Empfänger der Ware gegenüber, so geht dieser jenem vor.  \nArt. 926  \n1 Jeder Besitzer darf sich verbotener Eig enmacht mit Gewalt erweh ren. \n2 Er darf sich, wenn ihm die Sache durch Gewalt oder heimlich ent zo-\ngen wird, sofo

In [31]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=chroma_db.as_retriever())


In [32]:
response = chain.invoke(query)
print(response)

{'query': 'What is this document about?', 'result': 'This document appears to be excerpts from the Swiss Civil Code, specifically discussing various aspects of property law and obligations. Key topics covered include:\n\n1. **Schuldbrief (Mortgage Notes)**: \n   - The creation and nature of a Schuldbrief as a personal claim secured by a mortgage.\n   - The distinction between Register-Schuldbrief (registered mortgage notes) and Papier-Schuldbrief (paper mortgage notes).\n   - The rights and protections for both the creditor and debtor in relation to the Schuldbrief.\n\n2. **Transfer and Protection of Goods**: \n   - Rules regarding the transfer of goods represented by negotiable instruments issued by a carrier or warehouse (Art. 925).\n   - The precedence of rights of good-faith recipients of such documents or the goods themselves.\n\n3. **Defense Against Unlawful Possession**: \n   - The rights of a possessor to defend against unlawful actions (Art. 926).\n   - The immediate recapture

In [ ]:
def get_response(query:str):
    ## add the functionality to combine the functionalities above.